# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name: 

Student ID: 

Email: 

In [1]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [2]:
data = pd.read_csv("2008.csv.bz2")
data.describe(include="all")

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7009728,7.009728e+06,...,6.858079e+06,6.872670e+06,7.009728e+06,137434,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN,NaN,...,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1201754,NaN,...,NaN,NaN,NaN,54904,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,NaN,2.224200e+03,...,6.860852e+00,1.645305e+01,1.960618e-02,NaN,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,NaN,1.961716e+03,...,4.933649e+00,1.133280e+01,1.386426e-01,NaN,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,NaN,6.220000e+02,...,4.000000e+00,1.000000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,NaN,1.571000e+03,...,6.000000e+00,1.400000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,NaN,3.518000e+03,...,8.000000e+00,1.900000e+01,0.000000e+00,NaN,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01


In [3]:
#first question
print(data['CancellationCode'].value_counts()[:1])
#B is weather

B    54904
Name: CancellationCode, dtype: int64


In [4]:
#second question
print("Average distance", data['Distance'].mean())
print("Minimum distance", data['Distance'].min())
print("Maximum distance", data['Distance'].max())

Average distance 726.3870294253928
Minimum distance 11
Maximum distance 4962


In [5]:
#third question
f = data[data['Distance'] == data['Distance'].min()]
print(f['Year'], f['Month'], f['DayofMonth'])
for ind in f['FlightNum']:
    print(data[data['FlightNum'] == ind]['Distance'])

2547298    2008
4392215    2008
Name: Year, dtype: int64 2547298    5
4392215    8
Name: Month, dtype: int64 2547298    15
4392215    10
Name: DayofMonth, dtype: int64
151771     834
152430     834
153088     834
153545     834
154096     834
154750     834
155308     834
155962     834
156627     834
157287     834
157747     834
158301     834
158958     834
159514     834
160169     834
160833     834
161492     834
161950     834
162493     834
163151     834
163707     834
164361     834
165027     834
165688     834
166150     834
166704     834
167361     834
167917     834
168567     834
453560     834
          ... 
6776729    903
6776730    903
6776731    903
6776732    903
6776733    903
6776734    903
6776735    903
6776736    903
6776737    903
6776738    903
6776739    903
6776740    903
6776741    903
6776742    903
6776743    903
6776744    903
6776745    903
6776746    903
6776747    903
6776748    903
6776749    903
6776750    903
6776751    903
6776752    903
6776753

In [6]:
#fourth question
print(data['Origin'].value_counts()[:1])
print('Hartsfield-Jackson Atlanta International Airport')

ATL    414513
Name: Origin, dtype: int64
Hartsfield-Jackson Atlanta International Airport


In [7]:
#fifth question
group = data.groupby('Origin')
print(group['ArrTime'].mean())

Origin
ABE    1275.191819
ABI    1086.024847
ABQ    1485.039293
ABY    1264.013915
ACK    1380.863962
ACT    1346.037520
ACV    1340.364884
ACY    1307.508929
ADK    2141.663043
ADQ    1339.030166
AEX    1243.754331
AGS    1277.597540
AKN    1739.553571
ALB    1372.557040
ALO     794.567164
AMA    1361.533515
ANC    1230.692827
ASE    1380.380112
ATL    1538.640892
ATW    1320.374932
AUS    1429.958819
AVL    1300.312354
AVP    1317.197720
AZO    1256.796840
BDL    1379.090511
BET    1607.815293
BFL    1306.524848
BGM    1303.797143
BGR    1373.504498
BHM    1424.988602
          ...     
SPS    1402.234491
SRQ    1485.504576
STL    1488.664610
STT    1727.368765
STX    1945.861502
SUN    1356.823857
SUX    1438.864865
SWF    1383.226816
SYR    1334.538830
TEX    1450.150327
TLH    1333.159260
TOL    1163.566714
TPA    1490.027120
TRI    1256.122141
TUL    1347.662225
TUP    1242.400000
TUS    1395.074820
TVC    1241.551210
TWF    1366.456190
TXK    1266.499581
TYR    1269.875979
TYS  

In [15]:
dba = group.filter(lambda x: len(x) >= 1000).groupby('Origin').DepDelay.agg(lambda x: (x > 0).sum() / len(x))

In [20]:
print(dba.loc[dba == dba.max()], dba.max()) #Dallas airport

Origin
DAL    0.578642
Name: DepDelay, dtype: float64 0.5786418928942293


## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [ ]:
data = pd.read_csv('amazon.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# доля положительных примеров
data.ACTION.mean()

In [ ]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [21]:
from sklearn.neighbors import KNeighborsClassifier

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.